<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/AI_at_Scale/Assignments/Assignment2/performance_profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import regexp_replace, col, countDistinct, col, sum as spark_sum
from pyspark.sql.functions import count, when, lit, create_map
from pyspark.sql.types import StringType, DoubleType, IntegerType ,StructType,StructField
from pyspark.ml.feature import Imputer, VectorAssembler,StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import time
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor
from pyspark.ml import PipelineModel, Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [7]:
def create_spark_session(executor_cores="1", max_cores="2", executor_memory="1g"):
    return SparkSession.builder \
        .appName("Assignment2_ch24m571") \
        .config("spark.executor.cores", executor_cores) \
        .config("spark.cores.max", max_cores) \
        .config("spark.executor.memory", executor_memory) \
        .config("spark.driver.memory", "2g") \
        .getOrCreate()

def get_schema():
    return StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", IntegerType(), True),
    StructField("campaign", IntegerType(), True),
    StructField("pdays", IntegerType(), True),
    StructField("previous", IntegerType(), True),
    StructField("poutcome", StringType(), True),
    StructField("y", StringType(), True)

])

def run_single_performance_profiling(config):
    """Runs performance profiling for a single Spark configuration."""
    print(f"\nConfiguration: {config}")
    spark = create_spark_session(config["executor_cores"], config["max_cores"], config["executor_memory"])

    if config["parallelism"]:
        spark.conf.set("spark.sql.adaptive.coalescePartitions.parallelismFirst", "true")
        spark.conf.set("spark.default.parallelism", config["parallelism"])

    times = {}
    config_label = f"Cores: {config['executor_cores']}, Max: {config['max_cores']}, Mem: {config['executor_memory']}"
    if config['parallelism']:
        config_label += f", Parallelism: {config['parallelism']}"
    config_results = {'Configuration': config_label}

    try:
        start_time = time.time()
        data_path = "bank-full.csv"
        schema = get_schema()
        data = spark.read.csv(data_path, header=True, schema=schema, sep=';')
        train_df, test_df = data.randomSplit([0.8, 0.2], seed=42)
        times['Data Loading'] = time.time() - start_time
    except Exception as e:
        config_results['Data Loading Error'] = str(e)
        spark.stop()
        return config_results

    try:
        start_time = time.time()
        categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
        numerical_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

        indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_indexed", handleInvalid="keep") for col in categorical_cols]
        encoders = [OneHotEncoder(inputCol=f"{col}_indexed", outputCol=f"{col}_encoded") for col in categorical_cols]

        feature_cols = numerical_cols + [f"{col}_encoded" for col in categorical_cols]

        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
        label_indexer = StringIndexer(inputCol="y", outputCol="label")

        pipeline = Pipeline(stages=indexers + encoders + [assembler, label_indexer])
        preproc_model = pipeline.fit(train_df)
        train_df_processed = preproc_model.transform(train_df)
        test_df_processed = preproc_model.transform(test_df)
        times['Preprocessing'] = time.time() - start_time
    except Exception as e:
        config_results['Preprocessing Error'] = str(e)
        spark.stop()
        return config_results

    try:
        start_time = time.time()
        rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=10, seed=42)
        model = rf.fit(train_df_processed)
        times['Model Training'] = time.time() - start_time
    except Exception as e:
        config_results['Model Training Error'] = str(e)
        spark.stop()
        return config_results

    try:
        start_time = time.time()
        predictions = model.transform(test_df_processed)
        evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
        accuracy = evaluator.evaluate(predictions)
        times['Model Evaluation'] = time.time() - start_time
        times['Accuracy'] = accuracy
    except Exception as e:
        config_results['Model Evaluation Error'] = str(e)
        spark.stop()
        return config_results

    try:
        total_time = 0.0
        for k, v in times.items():
            if k != 'Accuracy' and isinstance(v, (int, float)):
                total_time += v
        times['Total Time'] = total_time
        config_results.update(times)
    except Exception as e:
        config_results['Total Time Calculation Error'] = str(e)

    spark.stop()
    return config_results

In [8]:
# Configuration 1
config1 = {"executor_cores": "1", "max_cores": "2", "executor_memory": "1g", "parallelism": None}
results1 = run_single_performance_profiling(config1)
print("\nResults for Configuration 1:")
print(pd.DataFrame([results1]).to_string(index=False))

# Configuration 2
config2 = {"executor_cores": "2", "max_cores": "2", "executor_memory": "1g", "parallelism": None}
results2 = run_single_performance_profiling(config2)
print("\nResults for Configuration 2:")
print(pd.DataFrame([results2]).to_string(index=False))

# Configuration 3
config3 = {"executor_cores": "2", "max_cores": "2", "executor_memory": "1g", "parallelism": "1"}
results3 = run_single_performance_profiling(config3)
print("\nResults for Configuration 3:")
print(pd.DataFrame([results3]).to_string(index=False))

# Configuration 4
config4 = {"executor_cores": "2", "max_cores": "2", "executor_memory": "1g", "parallelism": "2"}
results4 = run_single_performance_profiling(config4)
print("\nResults for Configuration 4:")
print(pd.DataFrame([results4]).to_string(index=False))


Configuration: {'executor_cores': '1', 'max_cores': '2', 'executor_memory': '1g', 'parallelism': None}

Results for Configuration 1:
            Configuration  Data Loading  Preprocessing  Model Training  Model Evaluation  Accuracy  Total Time
Cores: 1, Max: 2, Mem: 1g      0.133385      27.449638       16.747763          3.863088  0.893589   48.193874

Configuration: {'executor_cores': '2', 'max_cores': '2', 'executor_memory': '1g', 'parallelism': None}

Results for Configuration 2:
            Configuration  Data Loading  Preprocessing  Model Training  Model Evaluation  Accuracy  Total Time
Cores: 2, Max: 2, Mem: 1g      0.090955      12.321197        8.740624          3.033906  0.893589   24.186681

Configuration: {'executor_cores': '2', 'max_cores': '2', 'executor_memory': '1g', 'parallelism': '1'}

Results for Configuration 3:
                            Configuration  Data Loading  Preprocessing  Model Training  Model Evaluation  Accuracy  Total Time
Cores: 2, Max: 2, Mem: 1g, P